In [ ]:
import h5py
import os

# Path to your PRISMA .he5 file
file = "/home/daljit/hyperspec_project/data/PRS_L1_STD_OFFL_20201229101250_20201229101254_0001.he5"

print("File exists:", os.path.exists(file))

with h5py.File(file, "r") as f:
    print("\nTop-level groups:")
    for key in f.keys():
        print(" -", key)

    print("\nAll datasets inside file:\n")

    def show(name, obj):
        if isinstance(obj, h5py.Dataset):
            print("Dataset:", name, "→ shape:", obj.shape)

    f.visititems(show)


: 